In [9]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import openai
import os

In [2]:
openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
loader = WebBaseLoader("https://virtuelleakademie.ch/knowledge-base/")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                               chunk_overlap=10, 
                                               separators=["\n\n", "\n", " ", ""])



In [4]:
embedding = OpenAIEmbeddings()
persist_directory = 'docs/chroma/'

In [5]:
pages = loader.load()

In [6]:
all_splits = text_splitter.split_documents(pages)

In [7]:
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [8]:
vectordb.persist()

In [11]:
template = """Verwende die folgenden Kontextinformationen, um die Frage am Ende zu beantworten.
Wenn du die Antwort nicht weisst, sag einfach, dass du es nicht weisst -
versuche nicht, eine Antwort zu erfinden. Gib zusätzlich den Hinweis,
unbeantwortete Fragen an folgende Email zu schicken: email@virtuelleakademie.ch.
Antworte mit maximal 5 Sätzen und halte die Antwort so präzise wie möglich.
Sage immer "Danke für die Frage!" am Ende der Antwort. 
{context}
Frage: {question}
Antwort:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [13]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.4)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [14]:
question = "Welche Tools kann ich als Pinnwand verwenden?"

result = qa_chain({"query": question})
result["result"]

'Sie können das Tool "Padlet" als digitale Pinnwand verwenden. Es ermöglicht das Sammeln, Teilen und Organisieren von Inhalten in einem gemeinsamen Raum. Danke für die Frage!'